In [1]:
import os
import logging
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../secrets/.env.dev")
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [49]:
# Step 1: Read the main prompt template from the file
with open("prompt.txt", "r") as file:
    prompt_template = file.read()

# Step 2: Read the content for each placeholder from their respective files
with open("prd1_span.txt", "r") as file:
    prd_content = file.read()

with open("tdd1_span.txt", "r") as file:
    tdd_content = file.read()
    
with open("threat1.txt", "r") as file:
    threat_scenario = file.read()

with open("issues1.txt", "r") as file:
    issues_broken = file.read()

In [50]:
final_prompt = prompt_template.format(
    PRD_CONTENT=prd_content,
    TDD_CONTENT=tdd_content,
    THREAT_SCENARIO=threat_scenario,
    POTENTIAL_LAW_BROKEN=issues_broken
)

# Step 4: (Optional) Print the final, complete prompt to verify it's correct
print("--- FINAL PROMPT ---")
print(final_prompt)
print("--------------------")

--- FINAL PROMPT ---
You are Sentinel-Auditor, a world-class AI Trust & Safety analyst. Your mission is to determine if a
software feature's documented safeguards are sufficient to mitigate a specific, AI-generated
compliance risk scenario. You are meticulous, skeptical, and you prioritize user safety above all
else. You think step-by-step before reaching a conclusion.

You will be provided with the following information:

<PRD_CONTENT>
<span0>Product Requirements Document (PRD): Creator Connect</span0>
<span1>Document ID: PRD-2025-48B Title: PRD: Creator Connect (V1) Author: Priya Singh (Product Manager) Stakeholders: Creator Growth Team, Community Safety, Legal Target Launch: Q2 2025</span1>
<span2>1. Introduction & Problem Statement</span2>
<span3>One of the biggest challenges for new creators is the "cold start" problem—gaining initial visibility and navigating the platform's complexities. This leads to high churn in the first 30 days. Simultaneously, our established creators have 

In [51]:
import anthropic
anthropic_key = os.environ.get("ANTHROPIC_API_KEY")
client = anthropic.Anthropic(
    api_key=anthropic_key,
)
message = client.messages.create(
    model="claude-3-5-haiku-20241022",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": final_prompt}
    ]
)

In [52]:
message.content[0].text

'[\n  {\n    "title": "Insufficient Minor Protection in Mentorship Connections",\n    "evidence": ["PRD:span21", "TDD:span38", "TDD:span39"],\n    "reasoning": "The current safeguards do not provide adequate protection for minors. While the TDD mentions standard profanity and hate-speech filters, there are no specific protections against off-platform grooming attempts. The safety implementation relies solely on standard reporting mechanisms and CDS logging, which are reactive rather than preventative. Critically, there is no parental consent mechanism or age verification process for mentorship connections involving minors.",\n    "clarification_question": "Given the high-risk nature of creator-to-creator connections involving potential minors, what additional proactive safeguards will be implemented to prevent off-platform grooming and ensure parental consent, especially for users under 18?"\n  },\n  {\n    "title": "Weak Mentor Eligibility Verification",\n    "evidence": ["PRD:span19"

In [53]:
import json

python_object = json.loads(message.content[0].text)

with open("response.json", "w") as json_file:
    json.dump(python_object, json_file, indent=4)